In [1]:
import json
from src.constants import ROOT
from PIL import Image
from pathlib import Path
from tqdm.auto import tqdm

/home/ajd/miniconda3/envs/data-snapshot/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# To validate: reproduce images from json files
def extract_bboxes_from_annotation(json_path):
    with open(json_path, "r") as f:
        data = json.load(f)

    # Extract pdf file name, bboxes, and image paths
    bboxes = []
    for task in data:
        file = task["meta"].get("file")
        for annot in task["annotations"][0]["result"]:
            if annot["type"] == "rectanglelabels":
                annot["file"] = file
                idx = annot["item_index"]
                annot["page_image"] = task["data"]["pages"][idx]
                bboxes.append(annot)

    return bboxes


def extract_images_from_bboxes(bboxes, output_dir):
    for bbox in tqdm(bboxes):
        # Calculate pixel values
        values = bbox["value"]
        owidth = bbox["original_width"]
        oheight = bbox["original_height"]
        x1 = values["x"] / 100 * owidth
        x2 = x1 + values["width"] / 100 * owidth
        y1 = values["y"] / 100 * oheight
        y2 = y1 + values["height"] / 100 * oheight

        # Open image file and crop
        page_image_path = bbox["page_image"]
        data_path = str(ROOT / "labelstudio_data/") + "/"
        page_image_path = Path(page_image_path.replace("/data/local-files/?d=", data_path))
        img = Image.open(page_image_path).crop((x1, y1, x2, y2))

        # Save image to disk
        file = bbox["file"]
        idx = bbox["item_index"]
        kind = bbox["value"]["rectanglelabels"][0].lower().replace(" ", "")
        output_dir2 = Path(output_dir) / file
        output_dir2.mkdir(parents=True, exist_ok=True)
        fn = f"{file}_page_{idx:03d}_{kind}{page_image_path.suffix}"  # TODO: Handle repeated filenames / multiple same bbox in page
        img.save(output_dir2 / fn)


json_path = "project-19-at-2026-02-19-14-26-9aaf565b.json"
bboxes = extract_bboxes_from_annotation(json_path)
extract_images_from_bboxes(bboxes, "output/")

100%|███████████████████████████████████████████████████████████████████████████████████| 21/21 [00:05<00:00,  3.61it/s]


In [3]:
def extract_normalized_bboxes_from_annotation(json_path):
    pass